In [127]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import platform

In [128]:
import os
os.environ['PATH'] += ':/home/chandan/anaconda3/bin'
os.environ['SHELL'] = '/bin/bash'


In [129]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [130]:
openai=OpenAI()
model='gpt-4o'

In [131]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Ryzen5 processor. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [132]:
system_msg = "You are an assistant that takes python code and put time caluclation logic at suitable place."
system_msg = "So that user can estimate how much time the code is taking to run."

In [133]:
def user_prompt(python):
    prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    prompt += python
    prompt += "write it in proper executable cpp format don't write anything else not even comments and your code should be directly executable"
    return prompt

In [134]:
def python_format(python):
    prompt="Rewrite the python code and put the time calculation logic at a suitable place where required."
    prompt+="And print the run time after the actual output display in the format (Runtime of code: )"
    prompt+=python
    return prompt

In [135]:
def python_for(python):
    return [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": python_format(python)}
    ]

In [136]:
def message_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt(python)}
    ]

In [137]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [138]:
def optimize_gpt(python):
    stream=openai.chat.completions.create(model=model, messages=message_for(python), stream=True)
    reply=""
    for chunk in stream:
        frag=chunk.choices[0].delta.content or ""
        reply+=frag
        print(frag, end='', flush=True)
    write_output(reply)

In [139]:
def python_time(python_code):
    stream = openai.ChatCompletion.create(
        model=model,
        messages=python_for(python_code)
    )

    optimize_gpt(stream)

In [140]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [141]:
#!g++ -O3 -std=c++17 -march=x86-64 -o optimized optimized.cpp && ./optimized



In [142]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=model, messages=message_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [143]:
def optimize(python):
    result=stream_gpt(python)
    for stream_so_far in result:
        yield stream_so_far 

In [144]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()


In [145]:
def execute_cpp(code):
    write_output(code)
    
    compilers = ["clang++", "g++", "clang"]
    
    machine_arch = platform.machine().lower()

    compile_flags = ["-Ofast", "-std=c++17", "-o", "optimized", "optimized.cpp"]

    if machine_arch.startswith("arm"):
        compile_flags.extend(["-march=armv8-a", "-mcpu=generic"])
    else:
        compile_flags.append("-march=x86-64")

    for compiler in compilers:
        try:
            compile_cmd = [compiler] + compile_flags
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        
        except subprocess.CalledProcessError as e:
            print(f"Error using {compiler}: {e.stderr}")
            continue

    return "All compilers failed. Please check the code or environment."

In [146]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [147]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value="def hello():\n    print('Hello World!')", lines=10, placeholder="Type or paste Python code here")
        cpp = gr.Textbox(label="C++ code:", lines=10)
    
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

which: no node in (:/usr/bin:/home/chandan/anaconda3/bin:/home/chandan/anaconda3/bin:/home/chandan/anaconda3/bin:/home/chandan/anaconda3/bin)
